In [7]:
import torch
import torch.nn as nn

class OneDimensional_SE_ResNet(nn.Module):

    def __init__(self, block, blocks_num, num_classes=40):
        super(OneDimensional_SE_ResNet, self).__init__()
        self.in_channel = 64
        self.conv1 = nn.Conv1d(1, self.in_channel, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(self.in_channel)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(0.1)  
        self.conv2 = nn.Conv1d(self.in_channel, self.in_channel, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn2 = nn.BatchNorm1d(self.in_channel)
        self.layer1 = self._make_layer(block, 64, blocks_num[0])
        self.layer2 = self._make_layer(block, 128, blocks_num[1], stride=2)
        self.layer3 = self._make_layer(block, 256, blocks_num[2], stride=2)
        self.layer4 = self._make_layer(block, 512, blocks_num[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc1 = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

    def _make_layer(self, block, channel, block_num, stride=1):
        downsample = None
        if stride != 1 or self.in_channel != channel * block.expansion:
            downsample = nn.Sequential(
                nn.Conv1d(self.in_channel, channel * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(channel * block.expansion)
            )

        layers = [block(self.in_channel, channel, downsample=downsample, stride=stride)]
        self.in_channel = channel * block.expansion
        layers += [block(self.in_channel, channel) for _ in range(1, block_num)]

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x


class ResidualBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channel, out_channel, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=in_channel, out_channels=out_channel,
                               kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channel)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(0.1)
        self.conv2 = nn.Conv1d(in_channels=out_channel, out_channels=out_channel,
                               kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channel)
        self.downsample = downsample

        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(out_channel, out_channel // 16, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(out_channel // 16, out_channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv2(out)
        out = self.bn2(out)

        original_out = out
        b, c, _ = out.size()
        out = self.avg_pool(out).view(b, c)
        out = self.fc(out).view(b, c, 1)
        out = out * original_out

        out += identity
        out = self.relu(out)

        return out

def se_resnet34(num_classes=40):
    return OneDimensional_SE_ResNet(ResidualBlock, [3, 3, 3, 3], num_classes=num_classes)

In [8]:
import os
import json
import time
import torch
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np


def main():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    data_transform = transforms.Compose([ transforms.Grayscale(num_output_channels = 1),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5), (0.5))])

    # load npy
    npy_path = "../features/train_npy/Amazon Alexa Echo Dot 1/Amazon Alexa Echo Dot 1_0.npy"
    data = np.load(npy_path)
    data = Image.fromarray(data.astype(np.uint8))
    data = data_transform(data)
    
    # read class_indict
    json_path = '../idx_label.json'
    with open(json_path, "r") as f:
        class_indict = json.load(f)
    
    # create model
    model = se_resnet34(num_classes=40).to(device)
    # load model weights
    weights_path = "../TrimSENet_parameters.pth"
    model.load_state_dict(torch.load(weights_path, map_location=device))

    # prediction
    model.eval()
    with torch.no_grad():
        # predict class
        data = data.reshape(data.shape[0], 1, 1500)
        start_time = time.time()
        output = torch.squeeze(model(data.to(device))).cpu()
        predict = torch.softmax(output, dim=0)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"run time: {execution_time} s")
        predict_cla = torch.argmax(predict).numpy()

    print_res = "class: {}   prob: {:.3}".format(class_indict[str(predict_cla)],
                                                 predict[predict_cla].numpy())
    print(print_res)

if __name__ == '__main__':
    main()

run time: 0.012975215911865234 s
class: Amazon Alexa Echo Dot 1   prob: 1.0
